# sqlalchemy ORM

## Connect to database

In [1]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# Declare a Mapping

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users' 
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)  
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

In [3]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

## Create a Schema

In [4]:
Base.metadata.create_all(engine)

2017-04-16 12:17:03,895 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-16 12:17:03,900 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:03,903 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-16 12:17:03,905 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:03,908 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-16 12:17:03,910 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:03,914 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-04-16 12:17:03,915 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:03,917 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class¶

In [5]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

## Creating a Session

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

## Adding and Updating Objects¶

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
our_user = session.query(User).filter_by(name='ed').first()
ed_user is our_user

2017-04-16 12:17:03,943 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:03,946 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-16 12:17:03,948 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2017-04-16 12:17:03,951 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-04-16 12:17:03,952 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


True

In [8]:
session.commit()

2017-04-16 12:17:03,964 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
a = session.query(User).first()
a.password = '11111' 
session.add(a)
session.commit()
session.query(User).all()

2017-04-16 12:17:03,974 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:03,977 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
 LIMIT ? OFFSET ?
2017-04-16 12:17:03,978 INFO sqlalchemy.engine.base.Engine (1, 0)
2017-04-16 12:17:03,981 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-04-16 12:17:03,982 INFO sqlalchemy.engine.base.Engine ('11111', 1)
2017-04-16 12:17:03,984 INFO sqlalchemy.engine.base.Engine COMMIT
2017-04-16 12:17:03,985 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:03,987 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-04-16 12:17:03,988 INFO sqlalchemy.engine.base.Engine ()


[<User(name='ed', fullname='Ed Jones', password='11111')>]

## Rolling Back

In [10]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)
fake_user in session

True

In [11]:
session.rollback()
fake_user in session

2017-04-16 12:17:04,009 INFO sqlalchemy.engine.base.Engine ROLLBACK


False

In [12]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2017-04-16 12:17:04,020 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:04,023 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-04-16 12:17:04,025 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='11111')>]

## Query

In [13]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])
session.commit()

2017-04-16 12:17:04,037 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-16 12:17:04,039 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2017-04-16 12:17:04,041 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-16 12:17:04,042 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2017-04-16 12:17:04,043 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-16 12:17:04,045 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2017-04-16 12:17:04,046 INFO sqlalchemy.engine.base.Engine COMMIT


### Query a table

In [14]:
for i in session.query(User).order_by(User.id):
    print(i.name, i.fullname)

2017-04-16 12:17:04,053 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:04,057 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-04-16 12:17:04,058 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query by sepcific columns

In [15]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2017-04-16 12:17:04,072 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2017-04-16 12:17:04,075 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query the all rows and a column 

In [16]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2017-04-16 12:17:04,086 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-04-16 12:17:04,089 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='11111')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


### Change the label

In [17]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2017-04-16 12:17:04,100 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2017-04-16 12:17:04,105 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


### Make aliases

In [18]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2017-04-16 12:17:04,118 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias
2017-04-16 12:17:04,121 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='11111')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


### Slicing of the query

In [19]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2017-04-16 12:17:04,130 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2017-04-16 12:17:04,134 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


### `filter_by()`

In [20]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2017-04-16 12:17:04,145 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-16 12:17:04,147 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


### `filter()`

In [21]:
for name in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2017-04-16 12:17:04,157 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-16 12:17:04,159 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
('ed',)


### `AND` of filters

In [22]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2017-04-16 12:17:04,170 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2017-04-16 12:17:04,172 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', password='11111')>


# Building relationship between tables

In [23]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")    
Base.metadata.create_all(engine)

2017-04-16 12:17:04,197 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-16 12:17:04,199 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:04,200 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2017-04-16 12:17:04,201 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:04,203 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2017-04-16 12:17:04,204 INFO sqlalchemy.engine.base.Engine ()
2017-04-16 12:17:04,206 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses
jack.addresses = [
    Address(email_address='jack@google.com'),
    Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()

2017-04-16 12:17:04,219 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-16 12:17:04,221 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2017-04-16 12:17:04,225 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2017-04-16 12:17:04,226 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2017-04-16 12:17:04,227 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2017-04-16 12:17:04,229 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 5)
2017-04-16 12:17:04,231 INFO sqlalchemy.engine.base.Engine COMMIT


jack = session.query(User).filter_by(name='jack').one()
jack

In [25]:
jack.addresses

2017-04-16 12:17:04,238 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-16 12:17:04,241 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2017-04-16 12:17:04,242 INFO sqlalchemy.engine.base.Engine (5,)
2017-04-16 12:17:04,246 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2017-04-16 12:17:04,248 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

## Querying with Joins

### Implicit `join`

In [26]:
for u, a in session.query(User, Address).filter(User.id==Address.user_id).all():
    print(u, a)

2017-04-16 12:17:04,260 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id
2017-04-16 12:17:04,263 INFO sqlalchemy.engine.base.Engine ()
<User(name='jack', fullname='Jack Bean', password='gjffdd')> <Address(email_address='jack@google.com')>
<User(name='jack', fullname='Jack Bean', password='gjffdd')> <Address(email_address='j25@yahoo.com')>


### Automatic `join` by foreign key

In [27]:
session.query(User, Address).join(Address).all()

2017-04-16 12:17:04,273 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users JOIN addresses ON users.id = addresses.user_id
2017-04-16 12:17:04,277 INFO sqlalchemy.engine.base.Engine ()


[(<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='jack@google.com')>),
 (<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='j25@yahoo.com')>)]

### Explict condition

In [28]:
session.query(User, Address).join(Address, User.id==Address.user_id).all()

2017-04-16 12:17:04,291 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users JOIN addresses ON users.id = addresses.user_id
2017-04-16 12:17:04,294 INFO sqlalchemy.engine.base.Engine ()


[(<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='jack@google.com')>),
 (<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='j25@yahoo.com')>)]